# 02 - Data from the Web 
Kiran Bacsa, Samuel Beuret, and Valentine Santarelli

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.
* You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. 


Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import json 

import seaborn as sns

# set the default color palette
# try other options: 'Blues', sns.cubehelix_palette(8)
sns.set_palette('Blues')

# Seaborn can also use a context for different purpose
# possible values are paper, notebook, talk, and poster
sns.set_context("notebook")


import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import re

import warnings
warnings.filterwarnings('ignore')
from difflib import SequenceMatcher

##  200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). 

### Scrapping the web page 
Using Postman, part of the information needed was found  in a JSON file (name, rank, country, and region). The remaining information to scrap ranking was found using Beautiful Soup on each university page.By inspecting the web page, we found that the information to scrap was in the *div* of *class = faculty-main*. 
* number of faculty members international corresponds to the *div* of *class=inter faculty*
* number of faculty members total corresponds to the *div* of *class=total faculty*
* number of students international corresponds to the *div* of *class=total inter*
* number of students total corresponds to the *div* of *class=total student*

In [ ]:

# get name, rank, country and region of the 200 first universities

#url found using Postman
url='https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508005996450'
universities= requests.get(url).json()['data'][0:200] #take the first 200 universities
#first part of the dataframe for topuniversities.com
df_top1 = pd.DataFrame(universities)
df_top1 = df_top1.drop(['cc','core_id','guide', 'logo','score','stars','nid'], axis=1) #don't need this information

#list of urls to universities pages
urls =list(df_top1['url'])

# function to get the rest of the needed information on each university page in a dictionnary
def get_details(url):
    university= requests.get('https://www.topuniversities.com' + url)
    soup= BeautifulSoup(university.text,'html.parser')
    data= soup.find_all('div', class_= 'faculty-main') 

    # number of faculty members international
    if(soup.find_all('div', class_ = 'inter faculty') != []):
        inter_fac_mem = float(soup.find_all('div', class_ = 'inter faculty')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        inter_fac_mem = float('nan')
        
    # number of faculty members total
    if(soup.find_all('div', class_ = 'total faculty')!= []):
        total_fac_mem = float(soup.find_all('div', class_ = 'total faculty')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        total_fac_mem = float('nan')
        
    # number of students international
    if(soup.find_all('div', class_ = 'total inter')!= []):
        inter_student = float(soup.find_all('div', class_ = 'total inter')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        inter_student = float('nan')
        
    # number of students total
    if(soup.find_all('div', class_ = 'total student')!= []):
        total_student = float(soup.find_all('div', class_ = 'total student')[0].find_all('div', class_ = 'number')[0].string.replace(',',''))
    else:
        total_student = float('nan')

    #output dictionnary (keep the url to merge later)
    details = {'url' : url, '(QS) #faculty members international' : inter_fac_mem,
          '(QS) #faculty members total' : total_fac_mem,
          '(QS) #students international': inter_student,
          '(QS) #students total' : total_student}
    return details

#list of dictionnaries 
details = []    
for url in urls:
    details.append(get_details(url))
#convert to pd.DataFrame
df_top2 = pd.DataFrame(details)

#merge the two dataframes using the url
df_top= pd.merge(df_top1, df_top2, 'outer')
df_top.rename(columns={'rank_display': '(QS) rank', 'title': 'name'}, inplace=True)
df_top = df_top.drop('url', axis=1) #drop the url once the merge is done

df_top.head()


In order to avoid scrapping the page every time we need the dataset *df_top*, the later was exported in a file using the pandas fucntion *to_pickle()*, and imported with *from_pickle()*.

In [ ]:
#USE PICKLE TO SAVE DF
filename="./data/QS_ranking"
df_top.to_pickle(filename)

In [ ]:
#USE PICKLE TO LOAD DF
filename="./data/QS_ranking"
df_top =pd.read_pickle(filename)
df_top.head()

We verify that there is no tie in the ranking and if so we keep the order of display on the web page.
We found the ties using the *duplicated()* pandas function. Each time a duplicate is found (i.e a tie) we increment all the ranks below by one.
As expected the rank column is unique at the end.

In [ ]:
df_top['(QS) rank']=df_top['(QS) rank'].str.extract('(\d+)').astype('int64')
bool_dup = df_top['(QS) rank'].duplicated()
ind_dup = bool_dup[bool_dup == True].index

#until no more duplicates
for ind in ind_dup :
    #add +1 the all the ranks below this duplicate
    df_top['(QS) rank'][ind : len(df_top['(QS) rank'])] = df_top['(QS) rank'][ind : len(df_top['(QS) rank']) ] + 1;
    
print(df_top['(QS) rank'].is_unique)



### Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

In [ ]:
# adding the ratios to the df_top dataframe
df_top['(QS) faculty members/students'] = df_top['(QS) #faculty members total']/df_top['(QS) #students total']
df_top['(QS) international students/ total students'] = df_top['(QS) #students international']/df_top['(QS) #students total']
df_top.head()

In [ ]:
#Plotting the ratio faculty members/students for the 10 first universities
df_ratio1 = df_top.sort_values('(QS) faculty members/students', ascending =False) #sort by ratio
df_ratio1.set_index(['name'],inplace=True) #set name of universities as index for the plot
df_ratio1= df_ratio1['(QS) faculty members/students'] #extract only the column  needed

ax = df_ratio1[0:10].plot.bar()
ax.set_xlabel("Universities", fontsize=12)
plt.title("Ratio between faculty members and students")
plt.show()


The best universities in term of ratio of faculty members and students are :
* Caltech
* Yale University
* University of Oxford

In [ ]:
#Plotting the ratio of international students for the first 10 universities
df_ratio2 = df_top.sort_values('(QS) international students/ total students', ascending =False)#sort by ratio
df_ratio2.set_index(['name'],inplace=True) #set name of universities as index for the plot
df_ratio2= df_ratio2['(QS) international students/ total students']#extract only the column  needed

ax = df_ratio2[0:10].plot.bar()
ax.set_xlabel("Universities", fontsize=12)
plt.title("Ratio of international students")
plt.show()

The best universities in term of ratio of international students are :
* LSE
* EPFL
* Imperial College London



### Which are the best country in term of: (a) ratio between faculty members and students, (b) ratio of international students?

In [ ]:
#Create a new dataframe by aggregating values by country.
df_country = df_top.groupby('country').mean()
df_country.head()

In [ ]:
#sort by ratio and extract the columns needed
df_country_ratio1 = df_country.sort_values('(QS) faculty members/students', ascending =False)
df_country_ratio1= df_country_ratio1['(QS) faculty members/students']

#Plot the sorted countries
ax = df_country_ratio1[0:10].plot.bar()
ax.set_xlabel("Country", fontsize=12)
plt.title("Ratio between faculty members and students")
plt.show()

The best countries in term of ratio of faculty members and students are :
* Russia
* Denmark
* Saudi Arabia

In [ ]:
#sort by ratio and extract the columns needed
df_country_ratio2 = df_country.sort_values('(QS) international students/ total students', ascending =False)
df_country_ratio2= df_country_ratio2['(QS) international students/ total students']

#Plot the sorted countries
ax = df_country_ratio2[0:10].plot.bar()
ax.set_xlabel("Country", fontsize=12)
plt.title("Ratio of international students")
plt.show()

The best countries in term of ratio of international students are :
* UK
* Australia
* Switzerland

### Which are the best region in term of: (a) ratio between faculty members and students, (b) ratio of international students?

In [ ]:
#Create a new dataframe by aggregating values by region.
df_region = df_top.groupby('region').mean()
df_region.head()

In [ ]:
#Sort by ratio and extract the columns needed
df_region_ratio1 = df_region.sort_values('(QS) faculty members/students', ascending =False)
df_region_ratio1= df_region_ratio1['(QS) faculty members/students']

#Plot the sorted regions
ax = df_region_ratio1.plot.bar()
ax.set_xlabel("Region", fontsize=12)
plt.title("Ratio between faculty members and students")
plt.show()

The best regions in term of ratio of faculty members and students are :
* North America
* Asia
* Europe

In [ ]:
df_region_ratio2 = df_region.sort_values('(QS) international students/ total students', ascending =False)
df_region_ratio2= df_region_ratio2['(QS) international students/ total students']

ax = df_region_ratio2.plot.bar()
ax.set_xlabel("Region", fontsize=12)
plt.title("Ratio of international students")
plt.show()

The best regions in term of ratio of international students are :
* Oceania
* Europe
* North America

##  200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)).

### Scrapping the web page 

In [ ]:

url='https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

json_dict= requests.get(url).json()
raw_data = pd.DataFrame(json_dict['data'])
raw_data.columns
df_time=raw_data[['name','rank','location','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio']]

df_time['rank']=df_time['rank'].str.extract('(\d+)').astype('int64')
df_time = df_time.loc[df_time['rank']<201]
df_time.rename(columns={'location':'country','stats_number_students':'students','stats_pc_intl_students':'international students','stats_student_staff_ratio':'staff'},inplace=True)

df_time['students'] = df_time['students'].str.replace(',','').astype('int64')

df_time['international students'] = df_time['international students'].str.extract('(\d+)').astype('float64')
df_time['international students'] = (df_time['international students']/100)*df_time['students'].astype('float64')
df_time['international students'] = df_time['international students'].astype('int64')

df_time['staff'] = df_time['staff'].astype('float64')
df_time['staff'] = df_time['students'].astype('float64')/df_time['staff']
df_time['staff'] = df_time['staff'].astype('int64')

df_time.rename(columns={'rank': '(Times) rank', 'title': 'name','staff':'(Times) #faculty members total', 'students':'(Times) #students total','international students':'(Times) #students international' }, inplace=True)
df_time.head()

In [ ]:
#USE PICKLE TO SAVE DF
filename="./data/Times_ranking"
df_time.to_pickle(filename)

In [ ]:
#USE PICKLE TO LOAD DF
filename="./data/Times_ranking"
df_time =pd.read_pickle(filename)
df_time.head()

In [ ]:
df_time['(Times) faculty members/students']=df_time['(Times) #faculty members total'].astype('float64')/df_time['(Times) #students total']
barplot = df_time.sort_values('(Times) faculty members/students',ascending=False)['(Times) faculty members/students'].head(10).plot(kind='bar')

In [ ]:
df_time['(Times) international students/ total students']=df_time['(Times) #students international'].astype('float64')/df_time['(Times) #students total']
df_time.sort_values('(Times) international students/ total students',ascending=False).head()

In [ ]:
df_time.groupby(['country'])['(Times) faculty members/students'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_time.groupby(['country'])['(Times) international students/ total students'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
region = {'Asia':['China','Hong Kong','Japan','Singapore','South Korea','Taiwan']
          ,'Europe':['Austria','Belgium','Denmark','Finland','France','Germany','Ireland','Italy','Luxembourg','Netherlands','Norway','Russian Federation','Spain','Sweden','Switzerland','United Kingdom']
          ,'North America':['Canada','United States']
          ,'Africa':['South Africa']
          ,'Oceania':['Australia','New Zealand']}

region = { f : i for i in region for f in region[i] }

df_time['region'] = df_time['country'].map(region)

df_time.groupby(['region'])['(Times) faculty members/students'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_time.groupby(['region'])['(Times) international students/ total students'].mean().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_time.groupby(['region'])['(Times) rank'].count().sort_values(ascending=True).plot(kind='bar')

## Merging of the two DataFrames using university names. 
Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

One of the main problems that was observed was the fact that the Times ranking uses the universties' names expressed in their native language unlike the Qs-Ranking which uses the English name. The first step was therefore to clean our data by translating certain key words and caracters to their English equivalents. A specific translation was made for LMU Munich because its name in the QS-ranking was too distinct from its Times counterpart to be matched. Furthermore, redundant stop words that do not carry any relevant information such as 'The' and 'Of' were removed in order to make the university names more distinct.

In [ ]:
replace_target = ["É","é","&","Ludwig-Maximilians-Universität","-","Universite","Universität","Universitaet","Universitat","Technische","Autònoma","München"]
replace_with = ["E","e","and","LMU"," ","University","University","University","University","Technical","Autonomous","Munich"]
remove = ["The ","the ","Of ","of "]

for i in range(len(replace_target)):
    df_top['name'] = df_top['name'].str.replace(replace_target[i], replace_with[i])
    df_time['name'] = df_time['name'].str.replace(replace_target[i], replace_with[i])

for word in remove:
    df_top['name'] = df_top['name'].str.replace(word, "")
    df_time['name'] = df_time['name'].str.replace(word, "")


We can then define the functions that will be used to match the university names between each other. The first step in each function groups the universities by country. This operation considerably reduces the number of possible false positives during the matching phase. During this matching phase, for each country, we compare the similarity of the universities between themselves. If this similarity is above a predefined threshold, both universities are considered to be a match and their indexes are paired together. These universities are then removed from the symmetric difference datasets. The first function uses the SequenceMatcher similarity. This similarity is defined as 2.0\*M/T, where T is the total number of elements in both sequences, and M is the number of matches. The second function defines the similarity as the sum of the ratios of the number of common words versus the length of the universities' names (this function was mainly defined in order to compare similar names with different word orders). Both these functions return the dataframes containing the universities that haven't been paired up as well as the list of the paired up indexes.

In [ ]:
# match universities according to name similarity
def find_common_unmerged(df_top_out, df_time_out, threshold):
    n_duplicates = 0
    matched_top = []
    matched_time = []
    
    top_grouped = df_top_out.groupby('country')
    time_grouped = df_time_out.groupby('country')

    countries = pd.Series(list(set(df_top_out['country']).intersection(set(df_time_out['country']))))
    
    for country in countries:        
        unis_top = top_grouped.get_group(name=country)
        unis_time = time_grouped.get_group(name=country)     
        for name in unis_top['name']:
            for compare in unis_time['name']:
                score = SequenceMatcher(None, name, compare).ratio()
                if score > threshold:
                    n_duplicates = n_duplicates + 1
                    print('Matched ', name, ' with ', compare, ' with score ',score)
                    matched_top.extend(df_top_out.index[df_top_out['name'] == name])
                    matched_time.extend(df_time_out.index[df_time_out['name'] == compare])
                    
    print('Number of duplicates found : ', n_duplicates, 'out of ', len(df_top_out['name']), ' unmerged universities.')
    df_top_out = df_top_out.drop(matched_top, axis=0)
    df_time_out = df_time_out.drop(matched_time, axis=0)
    return df_top_out, df_time_out, matched_top, matched_time

In [ ]:
# match universities according to common substring
def find_common_unmerged_by_substrings(df_top_out, df_time_out, threshold):
    n_duplicates = 0
    matched_top = []
    matched_time = []
    
    top_grouped = df_top_out.groupby('country')
    time_grouped = df_time_out.groupby('country')

    countries = pd.Series(list(set(df_top_out['country']).intersection(set(df_time_out['country']))))
    
    for country in countries:        
        unis_top = top_grouped.get_group(name=country)
        unis_time = time_grouped.get_group(name=country)     
        for name in unis_top['name']:
            for compare in unis_time['name']:
                top_name_words = name.split()
                time_name_words = compare.split()
                common = set(top_name_words).intersection( set(time_name_words) )
                score = len(common)/len(top_name_words) + len(common)/len(time_name_words)
                if score > threshold:
                    n_duplicates = n_duplicates + 1
                    print('Matched ', name, ' with ', compare, ' with score ',score)
                    matched_top.extend(df_top_out.index[df_top_out['name'] == name])
                    matched_time.extend(df_time_out.index[df_time_out['name'] == compare])
                    
    print('Number of duplicates found : ', n_duplicates, 'out of ', len(df_top_out['name']), ' unmerged universities.')
    df_top_out = df_top_out.drop(matched_top, axis=0)
    df_time_out = df_time_out.drop(matched_time, axis=0)
    return df_top_out, df_time_out, matched_top, matched_time


We can now run these functions with different thresholds on the subdataframes that contain the symmetric differences of both datasets with respect to names.

In [ ]:
# get universities that weren't merged
df_top_out = df_top[~df_top['name'].isin(df_time['name'])]
df_time_out = df_time[~df_time['name'].isin(df_top['name'])]

total_matched_top = []
total_matched_time = []
m_top = []
m_time = []

# threshold : 0.9
df_top_out, df_time_out, m_top, m_time = find_common_unmerged(df_top_out, df_time_out, 0.9)
total_matched_top.extend(m_top)
total_matched_time.extend(m_time)
del m_top[:]
del m_time[:]

# threshold : 0.85
df_top_out, df_time_out, m_top, m_time = find_common_unmerged(df_top_out, df_time_out, 0.85)
total_matched_top.extend(m_top)
total_matched_time.extend(m_time)
del m_top[:]
del m_time[:]

# by substrings
df_top_out, df_time_out, m_top, m_time = find_common_unmerged_by_substrings(df_top_out, df_time_out, 1.4)
total_matched_top.extend(m_top)
total_matched_time.extend(m_time)
del m_top[:]
del m_time[:]


Now that we have detected the university pairs, we can change the Times ranking names to their QS ranking equivalents and merge both datasets.

In [ ]:
# replace time names with their top equivalent : 
for i in range(len(total_matched_time)):
    df_time.loc[total_matched_time[i],'name'] = df_top.loc[total_matched_top[i],'name']


In [ ]:
df_merged = pd.merge(df_top, df_time, how='outer')
df_merged.head()

## Exploratory analysis
* Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

In [ ]:
import seaborn as sns
#TO CHANGE LATER
# merge 'in' universities with same string name

df_merged = pd.merge(df_top, df_time, how='outer')
corr_pearson= df_merged.corr('pearson')

sns.heatmap(corr_pearson, 
            xticklabels=corr_pearson.columns.values,
            yticklabels=corr_pearson.columns.values)

 correlation  between
 
* faculty member international, faculty members total
* faculty member international, student international
* faculty member international, ratio fac/student
* faculty member international, international students (time)
* student international, ratio international students

In [ ]:
corr_spearman= df_merged.corr(method='spearman')

sns.heatmap(corr_spearman, 
            xticklabels=corr_spearman.columns.values,
            yticklabels=corr_spearman.columns.values)

In [ ]:
corr_kendall= df_merged.corr('kendall')

sns.heatmap(corr_kendall, 
            xticklabels=corr_kendall.columns.values,
            yticklabels=corr_kendall.columns.values)

In [ ]:
df_check= df_merged[[ '(QS) rank', '#faculty members total (QS)',
        '#students total (QS)',
       '#students international (QS)',
       'rank','staff', 'students',
       'international students']]
df_check = df_check.rename(columns={'QS rank': '(QS) rank','#faculty members total (QS)': '(QS) #faculty members total','#students total (QS)':'(QS) #students total', '#students international (QS)': '(QS) #students international' })
df_check = df_check.rename(columns={'rank': '(Times) rank ', 'staff': '(Times) #faculty members total','students':'(Times) #students total', 'international students':'(Times) #students international' });
corr_diff = df_check.corr();

test=corr_diff[['(QS) rank', '(QS) #faculty members total', '(QS) #students total',
       '(QS) #students international']].loc[
        '(Times) rank ':];
sns.heatmap(test)
            

# Defining the best university

* Can you find the best university taking in consideration both rankings? Explain your approach.

To determine the best university, we decided to use a Condorcet method. We group the univeristies two by two and we perform a vote (each university has a vote from one of the ranking if its score is higher than the other one). The overall score of the university is the number of vote it wins.

It has a drawback since it many of the univerities are tied in ranking. So we tried another to counter this effect by also considering the Shanghai ranking.

In [ ]:
#Parsing the dataset of the Shanghai ranking

# get name, rank, country and region of the 200 first universities



# function to get the rest of the needed information on each university page in a dictionnary
'''
universities= requests.get('http://www.shanghairanking.com/ARWU2017.html')
soup= BeautifulSoup(universities.text,'html.parser')
html_shanghai= soup.find_all('table', id="UniversityRanking")
rank_shanghai_even= html_shanghai[0].find_all('tr', class_="bgfd")
rank_shanghai_odd= html_shanghai[0].find_all('tr', class_="bgf5")

uni_list=[]
for uni in rank_shanghai_even[0:50]:
    uni_rank=int(uni.find('td').text);
    uni_name=uni.find('a').text
    uni_url='http://www.shanghairanking.com/' + uni.find('a')['href']; 
    uni_request= requests.get(uni_url)
    uni_soup= BeautifulSoup(uni_request.text,'html.parser')
    uni_country= uni_soup.find('div', class_="tab_content").find_all('td')[5].text
    uni_region= uni_soup.find('div', class_="tab_content").find_all('td')[3].text
    uni_list.append({'name' : uni_name, 'rank Shanghai' : uni_rank,
          'country' : uni_country,
          'region': uni_region})

    
for uni in rank_shanghai_odd[0:50]:
    uni_rank=int(uni.find('td').text);
    uni_name=uni.find('a').text
    uni_url='http://www.shanghairanking.com/' + uni.find('a')['href']; 
    uni_request= requests.get(uni_url)
    uni_soup= BeautifulSoup(uni_request.text,'html.parser')
    uni_country= uni_soup.find('div', class_="tab_content").find_all('td')[5].text
    uni_region= uni_soup.find('div', class_="tab_content").find_all('td')[3].text
    uni_list.append({'name' : uni_name, 'rank Shanghai' : uni_rank,
          'country' : uni_country,
          'region': uni_region})

df_shanghai=pd.DataFrame(uni_list)

df_shanghai=df_shanghai.sort_values('rank Shanghai', ascending=True)
df_shanghai.head()

df_shanghai.index.is_unique
df_shanghai.set_index('rank Shanghai')

#USE PICKLE TO SAVE DF
filename="./data/Shanghai_ranking"
df_shanghai.to_pickle(filename)
'''

In [ ]:
#loading the dataset

#USE PICKLE TO LOAD DF
filename="./data/Shanghai_ranking"
df_shanghai =pd.read_pickle(filename)
df_shanghai.head()

In [ ]:
#df_shanghai['name']=df_shanghai['name'].str.lower()
#df_shanghai=df_shanghai[['name','rank Shanghai']]



country_target=['USA','UK']
country_with=['United States','United Kingdom']

for i in range(len(country_target)):
    df_shanghai['country'] = df_shanghai['country'].str.replace(country_target[i], country_with[i])

for i in range(len(replace_target)):
    df_shanghai['name'] = df_shanghai['name'].str.replace(replace_target[i], replace_with[i])

for word in remove:
    df_shanghai['name'] = df_shanghai['name'].str.replace(word, "")
    
df_shanghai_out = df_shanghai[~df_shanghai['name'].isin(df_merged['name'])]
df_merged_out = df_merged[~df_merged['name'].isin(df_shanghai['name'])]

total_matched_merged = []
total_matched_shanghai = []
m_merged = []
m_shanghai = []

# threshold : 0.9
df_merged_out, df_shanghai_out, m_merged, m_shanghai = find_common_unmerged(df_merged_out, df_shanghai_out, 0.9)
total_matched_merged.extend(m_merged)
total_matched_shanghai.extend(m_shanghai)
del m_merged[:]
del m_shanghai[:]

# threshold : 0.85
df_merged_out, df_shanghai_out, m_merged, m_shanghai = find_common_unmerged(df_merged_out, df_shanghai_out, 0.85)
total_matched_merged.extend(m_merged)
total_matched_shanghai.extend(m_shanghai)
del m_merged[:]
del m_time[:]

# by substrings
df_merged_out, df_shanghai_out, m_merged, m_shanghai = find_common_unmerged_by_substrings(df_merged_out, df_shanghai_out, 1.49)
total_matched_merged.extend(m_merged)
total_matched_shanghai.extend(m_shanghai)
del m_merged[:]
del m_shanghai[:]

for i in range(len(total_matched_shanghai)):
    df_shanghai.loc[total_matched_shanghai[i],'name'] = df_merged.loc[total_matched_merged[i],'name']

df_shanghai=df_shanghai[['name','rank Shanghai']]
    
df_rank = pd.merge(df_merged, df_shanghai, how='outer')
df_rank=df_rank[['name','rank','QS rank','rank Shanghai']]
df_rank.columns=['name','TI rank','QS rank','SH rank']

In [ ]:


def compute_vote(rank_serie):
    rank_array = np.array(rank_serie)
    rank_array[np.isnan(rank_array)]=201
    rank_mat=np.tile(rank_array,(len(rank_array),1))
    rank_cmp=np.less_equal(rank_mat,np.transpose(rank_mat)).astype(int)
    return rank_cmp

In [ ]:
TI_cmp=compute_vote(df_rank['TI rank'])
SH_cmp=compute_vote(df_rank['SH rank'])
QS_cmp=compute_vote(df_rank['QS rank'])

In [ ]:
tot_mat=np.greater(QS_cmp+TI_cmp,1).astype(int)
df_rank['cond_score']= tot_mat.sum(axis=0)
df_rank.sort_values(['cond_score'],ascending=False).head(10)

We can see here the results of the Condorcet method considering only the Times and QS rankings, we can see that we got a unique best university (

In [ ]:
tot_mat=np.greater(QS_cmp+TI_cmp+SH_cmp,1.5).astype(int)
df_rank['cond_score']= tot_mat.sum(axis=0)
df_rank.sort_values(['cond_score'],ascending=False).head(10)